In [1]:
# Analysis on Covid-vaccine uptake/ Vaccine hesitancy
# Import libraries
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import numpy as np

In [2]:
data = pd.read_csv("data_no")
data.head(3)

,studysite,gender,age,marital_status,education,employed,diabetes,hypertension,heart_disease,high_cholestrol,...,tv_Radio,internet,social_media,hcp,family_members,government,pharmaceutical,scientific articles,don't_trust,VaccineStatus
0,faith,Female,27.0,Others,Undergraduate,Employed / Self Employed,Unchecked,Unchecked,Unchecked,Unchecked,...,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Checked,0
1,faith,Female,28.0,Single,Diploma,Employed / Self Employed,Unchecked,Unchecked,Unchecked,Unchecked,...,Unchecked,Unchecked,Unchecked,Checked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,0
2,faith,Female,26.0,Married,School Education,Unemployed,Unchecked,Unchecked,Unchecked,Unchecked,...,Checked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,Unchecked,0


In [3]:
data = data.drop(columns=['VaccineStatus'])

In [4]:
# set features to encode, by removing age and vaccine status columns
data_4_edit = data.drop(['age', 'studysite'], axis=1)

# features to encode
data_4_features = data_4_edit

# outcome
data_4_outcome = data[['age','studysite']]

In [5]:
from sklearn.preprocessing import OneHotEncoder

# Initialize OneHotEncoder
encoder = OneHotEncoder()

# Fit and transform the data
encoded_data_sparse = encoder.fit_transform(data_4_features)

# Convert sparse matrix to a dense array
encoded_data = encoded_data_sparse.toarray()

# Convert to DataFrame
encoded_df = pd.DataFrame(encoded_data, columns=encoder.get_feature_names_out(data_4_features.columns))

In [6]:
data_5 = pd.concat([encoded_df,data_4_outcome.reset_index(drop=True)], axis=1)

In [7]:
# group hospital names/ study site
study_site = {
    'private':1, 
    'faith':2,
    'public':3
}
data_5['studysite'] = data_5['studysite'].replace(study_site)
data_5['studysite'] = data_5['studysite'].astype('category')

C:\Users\Hp\AppData\Local\Temp\ipykernel_12232\2908225324.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  data_5['studysite'] = data_5['studysite'].replace(study_site)


In [8]:
# Correlation between variables 
correlation = data_5.corr()

corr = abs(correlation['studysite'])

# Select highest correlating values
co_Features = corr[corr>0.1]
co_Features.drop(labels=['studysite'], inplace=True)
co_Features.sort_values(ascending=False, inplace=True)
co_Features.head(20)

education_School Education              0.247204
job_worries_Unchecked                   0.243740
job_worries_Checked                     0.243740
free_citizens_Strongly Agree            0.238960
recommend_family_Disagree               0.212767
exposed_covid_Yes                       0.207391
exposed_covid_No                        0.207391
europe_america_Neutral                  0.206984
rapidly_developed_Disagree              0.200815
free_citizens_Disagree                  0.196818
financial_worries_Checked               0.190299
financial_worries_Unchecked             0.190299
pay_privately_No                        0.187324
free_citizens_Agree                     0.177388
hesitant_side_effects_Strongly Agree    0.171321
rapidly_developed_Strongly Agree        0.167127
scientific articles_Unchecked           0.164391
scientific articles_Checked             0.164391
safe_vaccine_Neutral                    0.163422
europe_america_Agree                    0.161137
Name: studysite, dty

In [9]:
# Prepare input/target variables
input = data_5.drop(['studysite'], axis=1)
target = data_5['studysite'].values

from sklearn.model_selection import train_test_split
X = input
y = target
X_train, X_test, y_train, y_test = train_test_split(X,y, test_size=0.3)

In [10]:
# SVM classifier
from sklearn import svm
clf = svm.SVC()
clf = clf.fit(X_train, y_train)

In [11]:
predictions = clf.predict(X_test)

In [12]:
# Performance metrics
from sklearn.metrics import accuracy_score
accuracy_score(y_test, predictions)

0.5813333333333334